## Fine Tune VGG for your dataset
We start with VGG-16 network pretrained on Imagenet Dataset for 1000 categories. Modify the last fully connected layer according to number of classes in custom dataset.

**Keras with Tensorflow as backend is used here.**

In order to train the model provide train and validation files. These file should be of the form

```train.txt``` <br>
```path/to/image1 label1```<br>
```path/to/image2 label2```<br>
```path/to/image3 label3```

In [1]:
#Starting with some cool imports
from keras.applications.vgg16 import (
    VGG16, preprocess_input, decode_predictions)
from keras.models import Sequential
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model
from keras import optimizers
from keras import utils
from keras.callbacks import ModelCheckpoint,TensorBoard


Using TensorFlow backend.


Get the pretrained VGG

In [ ]:
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg.